In [1]:
import pandas as pd  
import numpy as np
from glob import glob
from scipy.io import savemat, loadmat
from datetime import datetime, timedelta

import seaborn as sns
sns.reset_orig()
import pdb

import psycopg2
import sqlite3
import os
import gzip

from importlib import reload

import visualizeProfs as vp
%config InlineBackend.figure_format = 'retina'
from pchipOceanSlices import PchipOceanSlices

/home/tyler/anaconda3/envs/argo/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = other[key]
/home/tyler/anaconda3/envs/argo/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]
/home/tyler/anaconda3/envs/argo/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  self[key] = other[key]
/home/tyler/anaconda3/envs/argo/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]
/home/tyler/anaconda3/envs/argo/lib/python3.6/_collectio

# Init database

In [2]:
sqliteDb = "/storage/kakapo/JG_profiles.db"
tableName = "profiles"
conn = sqlite3.connect(sqliteDb)

In [3]:
conn.execute('''DROP TABLE IF EXISTS {};'''.format(tableName))

In [5]:
try:
    smdf = pd.read_sql_query('SELECT * FROM {} LIMIT 10;'.format(tableName), conn)
except Exception as err:
    print(err)
    pass

Execution failed on sql 'SELECT * FROM profiles LIMIT 10;': no such table: profiles


# Get reject list of profiles

In [6]:
rList = glob('/storage/kakapo/*temp*')
wl = glob('/storage/kakapo/*.xml')
header = ['platform_number', 'cycle', 'lat', 'lon', 'c1', 'c2', 'c3', 'c4', 'c5']

def get_lines(filename):
    with open(filename, "r") as f:
        data = f.readlines()
        lines = []
        for line in data: 
            line = ' '.join(line.split())
            line = line.split(' ')
            line[-1] = line[-1].replace('/n', '')
            lines.append(line)
        print(filename)
    return lines

def parse_kakapo(filename, header):
    '''parses reject and white list in kakapo'''
    lines = get_lines(filename)
    df = pd.DataFrame(data=lines, columns=header)
    df['profile_id'] = df['platform_number'].apply(lambda x: str(x)) + '_'  + df['cycle'].apply(lambda x: str(x))
    return df

rejectDf = parse_kakapo(rList[0], header)
whiteList = parse_kakapo(wl[0], header)

/storage/kakapo/reject_cycle_fromvar_temp_01x01_finl
/storage/kakapo/ar_whitelist_2019.xml


In [7]:
print(rejectDf.shape)
rejectDf.head()

(2135, 10)


,platform_number,cycle,lat,lon,c1,c2,c3,c4,c5,profile_id
0,6903190,18,-60.917,304.997,0,2.154,0.257,6.0xtimes1975.0,None,6903190_18
1,5903417,88,-37.717,332.474,1,0.343,0.051,6.0xtimes1975.0,None,5903417_88
2,6901697,81,-37.674,332.432,1,0.551,0.051,6.0xtimes1975.0,None,6901697_81
3,3900529,66,-37.284,285.835,1,0.230,0.038,6.0xtimes1975.0,None,3900529_66
4,5904120,53,-36.369,330.982,1,0.640,0.048,6.0xtimes1975.0,None,5904120_53


In [8]:
ok_ids = whiteList['profile_id'].tolist()

In [9]:
rejectDf = rejectDf[~rejectDf['profile_id'].isin(ok_ids)]

In [10]:
print(rejectDf.shape)
rejectDf.head()

(1113, 10)


,platform_number,cycle,lat,lon,c1,c2,c3,c4,c5,profile_id
0,6903190,18,-60.917,304.997,0,2.154,0.257,6.0xtimes1975.0,None,6903190_18
1,5903417,88,-37.717,332.474,1,0.343,0.051,6.0xtimes1975.0,None,5903417_88
2,6901697,81,-37.674,332.432,1,0.551,0.051,6.0xtimes1975.0,None,6901697_81
3,3900529,66,-37.284,285.835,1,0.230,0.038,6.0xtimes1975.0,None,3900529_66
4,5904120,53,-36.369,330.982,1,0.640,0.048,6.0xtimes1975.0,None,5904120_53


# Parse data files

In [ ]:
files = glob('/storage/kakapo/*_padj.dat.gz')
header = ['latitude', 'longitude', 'day', 'month', 'year', 'hr', 'min',
          'sec', 'pressure', 'temperature', 'potential temperature',
          'salinity', 'DMODE status', 'WMO', 'cycle number', 'original netcdf file']

def get_gzip_lines(filename):
    with gzip.open(file, "r") as f:
        data = f.readlines()
        lines = []
        for line in data:
            line = line.decode('utf-8')
            line = ' '.join(line.split())
            line = line.split(' ')
            line[-1] = line[-1].replace('/n', '')
            lines.append(line)
        print(filename)
    return lines


#conn.execute('''DROP TABLE IF EXISTS {};'''.format(tableName))
for file in files:
    lines = get_gzip_lines(file)
    df = pd.DataFrame(data=lines, columns=header)
    df['profile_id'] = df['WMO'].apply(lambda x: str(x)) + '_'  + df['cycle number'].apply(lambda x: str(x))
    df.to_sql(tableName, conn, if_exists='append', index=False)

/storage/kakapo/200401_padj.dat.gz
/storage/kakapo/200402_padj.dat.gz
/storage/kakapo/200403_padj.dat.gz
/storage/kakapo/200404_padj.dat.gz
/storage/kakapo/200405_padj.dat.gz
/storage/kakapo/200406_padj.dat.gz
/storage/kakapo/200407_padj.dat.gz
/storage/kakapo/200408_padj.dat.gz
/storage/kakapo/200409_padj.dat.gz
/storage/kakapo/200410_padj.dat.gz
/storage/kakapo/200411_padj.dat.gz
/storage/kakapo/200412_padj.dat.gz
/storage/kakapo/200501_padj.dat.gz
/storage/kakapo/200502_padj.dat.gz
/storage/kakapo/200503_padj.dat.gz
/storage/kakapo/200504_padj.dat.gz
/storage/kakapo/200505_padj.dat.gz
/storage/kakapo/200506_padj.dat.gz
/storage/kakapo/200507_padj.dat.gz
/storage/kakapo/200508_padj.dat.gz
/storage/kakapo/200509_padj.dat.gz
/storage/kakapo/200510_padj.dat.gz
/storage/kakapo/200511_padj.dat.gz
/storage/kakapo/200512_padj.dat.gz
/storage/kakapo/200601_padj.dat.gz
/storage/kakapo/200602_padj.dat.gz
/storage/kakapo/200603_padj.dat.gz
/storage/kakapo/200604_padj.dat.gz
/storage/kakapo/2006

In [4]:
smdf = pd.read_sql_query('SELECT * FROM {} LIMIT 10;'.format(tableName), conn)

DatabaseError: Execution failed on sql 'SELECT * FROM profiles LIMIT 10;': no such table: profiles

In [61]:
conn.execute('''SELECT COUNT(*) FROM {};'''.format(tableName))

(553930, 17)